In [110]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd

import json

api_key = 'RGAPI-2bf73814-83af-4540-a2e0-46eee1c8271f'
watcher = LolWatcher(api_key=api_key)
my_region = 'na1'
me = watcher.summoner.by_name(my_region, 'yENatividad')

In [111]:
def clean_dict_mask(dic):
    keys_to_remove = []
    for k,v in dic.items():
        if type(v) is dict:
            keys_to_remove = keys_to_remove + [k]
    return keys_to_remove

## Match Summary

In [112]:
my_matches = watcher.match.matchlist_by_puuid(my_region, me['puuid'])
latest_match = watcher.match.by_id(my_region, my_matches[0])

In [113]:
players_list = latest_match['info']['participants']
rows = []

# Create mask for non-numerical dict fields 
keys_to_remove = clean_dict_mask(players_list[0])

# Construct dictionary list that prepares for pandas DataFrame
for player in players_list:
    for key in keys_to_remove:
        if key in player.keys():
            player.pop(key)
    rows += [player]

# Pandas Dataframe
match_summary = pd.DataFrame(rows)
match_summary 

,assists,baronKills,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,damageDealtToBuildings,...,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,9,1,0,20440,18,50,Swain,0,2,0,...,4000,0,0,11,0,10,0,1,9,False
1,14,0,0,18112,17,33,Rammus,0,13,0,...,2614,0,0,11,0,35,8,4,13,False
2,8,0,0,20408,18,105,Fizz,0,2,3070,...,1854,1,1,11,0,22,0,1,14,False
3,6,0,0,23534,18,119,Draven,0,5,0,...,603,0,0,11,0,33,3,2,13,False
4,14,0,0,18179,17,89,Leona,0,13,0,...,1244,0,0,11,0,69,9,9,35,False
5,8,0,1,20936,18,82,Mordekaiser,0,6,5423,...,848,2,3,1,0,38,4,11,5,True
6,14,1,1,27356,18,102,Shyvana,0,6,1166,...,1991,2,2,1,0,43,2,11,3,True
7,13,0,2,19674,18,99,Lux,0,2,3853,...,392,1,5,1,0,27,0,6,10,True
8,16,0,0,18346,17,81,Ezreal,0,3,9760,...,2241,6,7,1,0,15,0,3,9,True
9,26,0,0,16289,16,267,Nami,0,13,2201,...,613,0,6,1,0,100,11,9,46,True


## Match Timeline 10min

In [119]:
latest_timeline = watcher.match.timeline_by_match(my_region, my_matches[0])

In [120]:
time_frame = 15

players_list = list(latest_timeline['info']['frames'][time_frame]['participantFrames'].values()).copy()
rows = []

# Construct intermediate dictionary list
# Expand the `damageStats` field and merge to the player's dictionary
for player in players_list:
    damage_stat = player['damageStats']
    player.update(damage_stat)

# Create mask for non-numerical dict fields
keys_to_remove = clean_dict_mask(players_list[0])

# Construct dictionary list that prepares for pandas DataFrame
for player in players_list:
    for key in keys_to_remove:
        if key in player.keys():
            player.pop(key)
    rows += [player]

# Pandas Dataframe
ten_min_summary = pd.DataFrame(rows)
ten_min_summary

,currentGold,goldPerSecond,jungleMinionsKilled,level,minionsKilled,participantId,timeEnemySpentControlled,totalGold,xp,magicDamageDone,...,magicDamageTaken,physicalDamageDone,physicalDamageDoneToChampions,physicalDamageTaken,totalDamageDone,totalDamageDoneToChampions,totalDamageTaken,trueDamageDone,trueDamageDoneToChampions,trueDamageTaken
0,663,0,0,10,77,1,160896,4088,6999,30069,...,4721,6229,796,2099,39819,3905,7244,3520,0,423
1,400,0,61,8,1,2,312432,5325,4676,28173,...,3388,12376,915,7208,44880,3987,10890,4330,328,294
2,149,0,0,11,97,3,119520,7174,7446,35465,...,2381,9174,2147,3321,45213,8933,6223,574,574,520
3,1329,0,0,9,96,4,19913,5229,5186,75,...,469,35024,4975,4823,37485,5019,5491,2385,0,198
4,460,30,0,7,22,5,20351,3210,3979,2103,...,2798,3270,486,3860,9020,1514,6807,3646,356,148
5,302,0,0,11,108,6,19852,6452,8278,39393,...,4285,10102,733,3141,50105,7409,7427,609,499,0
6,241,0,99,9,10,7,281753,5581,5761,48730,...,3344,21147,1019,7975,78436,3059,11713,8558,338,394
7,818,0,0,9,61,8,164376,3728,5451,21934,...,4467,6199,666,1981,31698,2537,6677,3564,214,228
8,383,0,0,9,116,9,58282,6366,5319,5724,...,1283,46806,8182,3319,55148,10647,4869,2618,532,266
9,155,30,0,6,19,10,59324,3430,3166,2285,...,1173,2764,550,3514,6949,2077,5057,1899,0,370


In [118]:
ten_min_summary.columns

Index(['currentGold', 'goldPerSecond', 'jungleMinionsKilled', 'level',
       'minionsKilled', 'participantId', 'timeEnemySpentControlled',
       'totalGold', 'xp', 'magicDamageDone', 'magicDamageDoneToChampions',
       'magicDamageTaken', 'physicalDamageDone',
       'physicalDamageDoneToChampions', 'physicalDamageTaken',
       'totalDamageDone', 'totalDamageDoneToChampions', 'totalDamageTaken',
       'trueDamageDone', 'trueDamageDoneToChampions', 'trueDamageTaken'],
      dtype='object')